In [2]:
import os
from os.path import join
#import img2pdf
import re
from tqdm import tqdm
from file_function import write_log_csv_upload
from PIL import Image, ImageSequence

### pillow 여러 폴더 작업

In [17]:
#### csv읽어서 재작업
# import pandas as pd
# df = pd.read_csv(r"D:\업로드완료\광주 모음 대출원장 티와이1\20221117 175127_319_파일병합 실패.csv", header=None)
# df.columns = ["root", "dir", "1"]
# df.dir = df.dir.astype("str")
# dir_list = list(df.dir)
# path = r"D:\업로드완료\광주 모음 대출원장 티와이1"
# print(dir_list[0], type(dir_list[0]))
# for d in dir_list :
#     temp = [x.name for x in os.scandir(join(path, d)) if x.is_file()]
#     if len(temp) != 0 :
#         print(d)

20407804 <class 'str'>


In [5]:
#### 전처리 
# 이미지 아닌 파일 있는지 확인
path = r"D:\카카오\법원문건(개인회생)"
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
for r, ds, fs in os.walk(path) :
    for f in fs :
        if p_extension.search(f) == None :
            print(f)

In [6]:
############################################
path = r"D:\카카오\법원문건(개인회생)"
path_out = path
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
addInfo = "카카오 법원문건 B imgTopdf"
add_file_name = "개인회생 매각사문건"
############################################
path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'
path_fail_log = path

success = []
fail = []
notImg = []
empthFolder = 0

dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)

##### 작업단뒤 : 디렉토리
### 이미지들의 경로를 리스트에 저장하기
for d in tqdm(dir_list, total=total) :
    path_img_list = []
    file_list = [x.name for x in os.scandir(join(path, d)) if x.is_file()]
    for f in file_list :
        if p_extension.search(f) :
            ########################### 짝수페이지 버리기
            # num = int(re.search(r"(?<=_)\d{4,4}", f).group())
            # if num % 2 != 0 :
            ###########################
            path_img_list.append(join(path, d, f)) # 전체경로를 저장해야 함
            # else : continue
        else :
            notImg.append([path, d, f])
            continue

    if len(path_img_list) > 0 : #이미지 파일이 하나 이상 있는 경우만 
    ### 저장할 경로 및 pdf파일명 설정
        new_name = d +" "+ add_file_name + ".pdf" ###############
        output = join(path_out, new_name)
        sn = 0
        while os.path.exists(output) :
            sn += 1
            new_name = d + " " + add_file_name+ " " + str(sn) + ".pdf"
            output = join(path, new_name)

        ### 리스트에서 이미지를 하나씩 읽어서 컨버트 및 통합pdf만들기
        images = []
        for path_image in path_img_list :
            
            image = Image.open(path_image)   # 1. 이미지 읽기
            
            for i, page in enumerate(ImageSequence.Iterator(image)): 
                page = page.convert("RGB") # 2. 이미지 convert
                images.append(page) # 3. convert된 이미지를 리스트에 저장
                # Image.Image.close(image) 소용이 없네?
        try :
            if len(images) == 1: # 4. images[0]을 저장. save_all과 append_images 속성으로 다음페이지 붙여 저장
                images[0].save(output)
            else:
                images[0].save(output, save_all=True,append_images=images[1:]) # pdf_path 는 마지막 tiff파일명으로 대체
            success.append([path, d, output])
        except Exception as e :
            fail.append([path, d, e.__class__])
    else :
        empthFolder += 1

loss = total - len(success) -len(fail) - empthFolder # 폴더 수 - PDF만든거 - pdf못 만든거
print(loss, "개 누수")
print(f"전체 : {total}개, 파일변환 : {len(success)}개, 실패{len(fail)}개, 빈폴더{empthFolder}개")
print(f"{len(notImg)}개 이미지 아닌 파일 있음")

write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path_fail_log, "파일병합 실패")

100%|██████████| 1324/1324 [58:17<00:00,  2.64s/it] 


0 개 누수
전체 : 1324개, 파일변환 : 1324개, 실패0개, 빈폴더0개
0개 이미지 아닌 파일 있음
log 파일이 비어있습니다. path : D:\카카오\법원문건(개인회생)


In [1]:
notImg

NameError: name 'notImg' is not defined

##### 통합 완료한 img 파일 지우기/옮기기

In [2]:
# 지우기
############################################
path = r"D:\구 스캔파일\광주 공유폴더\모음\대출원장\티와이머니_2 원인서류 이미지"
path_dst_base = r"D:\업로드완료\광주 모음 대출원장 티와이_2원인 img파일"
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
addInfo = "광주 모음 대출원장 티와이_2원인 img삭제"
#add_file_name = "A"
############################################
path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'
path_fail_log = path
success, fail = [], []

dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)
pdf_count = 0
for d in tqdm(dir_list, total=total) :
    path_img_list = []
    file_list = [x.name for x in os.scandir(join(path, d)) if x.is_file()]
    add_pdf_count = False
    for f in file_list :
        if p_extension.search(f) :
            add_pdf_count = True
            try :
                # os.remove(join(path, d, f))
                dst_dir = join(path_dst_base, d, "B") #################
                if not os.path.exists(dst_dir) :
                    os.makedirs(dst_dir)
                os.rename(join(path, d, f), join(dst_dir, f))
                success.append([path, d, f])
            except Exception as e :
                fail.append([path, d, f, e.__class__])
                pass
    if add_pdf_count :
        pdf_count += 1

print(f"{len(success)}개 이미지 파일 삭제, {len(fail)}개 이미지 파일 삭제 실패")
print(f"pdf파일은 총 {pdf_count}개여야 함")

write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path_fail_log, "img파일 삭제 실패")

100%|██████████| 928/928 [18:47<00:00,  1.22s/it] 


151237개 이미지 파일 삭제, 0개 이미지 파일 삭제 실패
pdf파일은 총 902개여야 함
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\모음\대출원장\티와이머니_2 원인서류 이미지


### pillow 단일 폴더 작업

In [2]:
# 폴더내 이미지 파일 하나로 합치기. path에 생성
############################################
path = r"D:\구 스캔파일\광주 공유폴더\모음\정리할것\중복확인필요\중복건확인필요\티와이머니1차 양통\pdf로변환"
add_file_name = path.split(os.path.sep)[-1] + " 원인서류 여타매각사자료포함"
############################################
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 

img_list = [join(path, f) for f in os.listdir(path) if p_extension.search(f)]

output = join(path, add_file_name + ".pdf")
if os.path.exists(output) :
    sn = 0
    while os.path.exists(output) :
        sn += 1
        output = join(path, add_file_name + str(sn) + ".pdf")

images = []
for path_image in img_list :
    
    image = Image.open(path_image)   # 1. 이미지 읽기
    
    for i, page in enumerate(ImageSequence.Iterator(image)): 
        page = page.convert("RGB") # 2. 이미지 convert
        images.append(page) # 3. convert된 이미지를 리스트에 저장

if len(images) == 1: # 4. images[0]을 저장. save_all과 append_images 속성으로 다음페이지 붙여 저장
    images[0].save(output)
else:
    images[0].save(output, save_all=True,append_images=images[1:]) # pdf_path 는 마지막 tiff파일명으로 대체

ValueError: unknown file extension: 

### 단일 파일을 pdf로 변환

In [4]:
############################################
path = r"D:\구 스캔파일\광주 공유폴더\모음\정리할것\중복확인필요\중복건확인필요\티와이머니1차 양통\pdf로변환"
path_dst_dir = join(path, "pdf")
add_file_name = ""
############################################
success, fail = [], []
if not os.path.exists(path_dst_dir) :
    os.mkdir(path_dst_dir)

p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 

file_list = [f.name for f in os.scandir(path) if p_extension.search(f.name)]
total = len(file_list)
for f in tqdm(file_list, total=total) :
    stem = os.path.splitext(f)[0]
    try :
        img = Image.open(join(path, f))
        img_rgb = img.convert('RGB')
        img_rgb.save(join(path_dst_dir, stem + '.pdf'))
        success.append([path, f, path_dst_dir, stem])
    except Exception as e:
        fail.append([path, f, e.__class__, e])

print(f"누수 파일 : {total-len(success)-len(fail)}개")
print(f"{total}개 중 {len(success)}개 성공, {len(fail)}개 실패")

100%|██████████| 38/38 [00:02<00:00, 18.36it/s]

누수 파일 : 0개
38개 중 38개 성공, 0개 실패


### pdf파일 페이지 쪼개기

In [ ]:
# step1.관련 모듈 import
from PyPDF2 import PdfFileReader, PdfFileWriter

# step2.기존 PDF 불러오기
path = r"C:\Users\SL\Desktop\test"
file = "1.pdf"
pdfReader = PdfFileReader(join(path, file), "rb")

# 페이지 하나씩 받아와서 저장하는 반복문
for pageNum in range(pdfReader.numPages):
    
    # step3.새로 만들 pdf 객체 생성 (계속 누적되지 않기 위해 for문 안으로 넣음)
    pdfWriter = PdfFileWriter()
    
    # step4.기존 PDF에서 한 페이지씩 가져오기
    page = pdfReader.getPage(pageNum)
    
    # step5.위에서 가져온 페이지를 새로 만든 PDF에 붙여넣기
    pdfWriter.addPage(page)

    # step6.새로운 PDF 파일을 해당 경로('./')에 원하는 이름으로 저장
    # (이름을 계속 다르게 해주기 위해서 f 문자열 포매팅 개념을 이용)
    pdfWriter.write(open(join(path, f"분할한 PDF 파일 {pageNum+1}.pdf"), "wb"))

### img2pdf - 에러나고 느리고 출력도 이상하니 쓰지마

In [35]:
# ### 여러장 짜리 이것도 됨
# ############################################
# path = r"D:\구 스캔파일\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류"
# path_out = path
# p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
# add_file_name = " 원인서류 (여타매각사자료통합)"
# ############################################
# path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'
# path_fail_log = path

# success = []
# fail = []
# notImg = []

# dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
# total=len(dir_list)

# for d in tqdm(dir_list, total=total) :
#     temp = []
#     file_list = os.listdir(join(path, d))
#     for f in file_list :
#         if p_extension.search(f) :
#             ########################### 짝수페이지 버리기
#             # num = int(re.search(r"(?<=_)\d{4,4}", f).group())
#             # if num % 2 != 0 :
#             ###########################
#             temp.append(join(path, d, f)) # 전체경로를 저장해야 함
#             # else : continue
#         else :
#             notImg.append([path, d, f])
#             continue
    
#     if len(temp)>0 :
#         try :
#             output = join(path_out, d + add_file_name + ".pdf")
#             if os.path.exists(output) :
#                 sn = 0
#                 while os.path.exists(output) :
#                     sn += 1
#                     output = join(path_out, d + add_file_name + str(sn) + ".pdf")

#             with open(output,"wb") as f:
#                 f.write(img2pdf.convert(temp))

#             success.append([d, output])
#         except Exception as e :
#             fail.append([path, d, e.__class__])

# loss = total - len(success) -len(fail) # 폴더 수 - PDF만든거 - pdf못 만든거
# print(loss, "개 누수")
# print(f"전체 : {total}개, 파일변환 : {len(success)}개, 실패{len(fail)}개")
# print(f"{len(notImg)}개 이미지 아닌 파일 있음")

# write_log_csv_upload(success, path_success_log, "파일병합 성공")
# write_log_csv_upload(fail, path_fail_log, "파일병합 실패")

  0%|          | 0/303 [00:00<?, ?it/s]

setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 482864
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 470424
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 469133
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 460491
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 484380
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 483060
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 484720
setting <module 'PIL.TiffImagePlugin' from 'c:\\

 14%|█▎        | 41/303 [04:41<1:19:35, 18.23s/it]

setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 403180
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 401850
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 401280


 27%|██▋       | 83/303 [04:44<00:30,  7.13it/s]  

...

setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 481605
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 484512
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 484806
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 485100
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 484806


 96%|█████████▌| 291/303 [08:29<00:01,  7.62it/s]

setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 115039
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 482384
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 484720
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 481023
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 481314
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 481800
setting <module 'PIL.TiffImagePlugin' from 'c:\\Users\\sl\\anaconda3\\lib\\site-packages\\PIL\\TiffImagePlugin.py'>.STRIP_SIZE = 482384
setting <module 'PIL.TiffImagePlugin' from 'c:\\

100%|██████████| 303/303 [09:43<00:00,  1.92s/it]

27 개 누수
전체 : 303개, 파일변환 : 246개, 실패30개
0개 이미지 아닌 파일 있음


In [37]:
cnt = 0
for root, dirs, files in os.walk(path) :
    for dir in dirs :
        try :
            os.rmdir(join(root, dir))
            cnt += 1
        except :
            pass
print(cnt)

27


In [19]:
print(len(success), len(fail))

0 7


In [7]:
print(*notImg, sep="\n")

['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498815 이갑선 원인서류', '0220141270521_신용대출신청서_0001_61923136.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498815 이갑선 원인서류', '0220161031700_신용대출신청서_0001_66553331.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498832 전현철 원인서류', '전현철_06775710058933_.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498841 임수영 원인서류', '049783386720004_임수영.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498841 임수영 원인서류', '4101090007240295_임수영.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498868 김명식 원인서류', '10055793106-1.pdf']
['D:\\extraKe

In [41]:
# 이미지 파일 아닌 거 처리하기
new = notImg[127:]

In [10]:
for x in notImg : 
    if not re.search("pdf|PDF", x[2]) :
        print(x)

['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20499526 김광현 원인서류', '19031100023_녹취파일.wav']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20500043 고영세 원인서류', '4347014100036.wav']


In [15]:
dst_dir = r"D:\extraKey\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\22.베리타스_솔림헬프\원인서류 notImg"
success = []
fail = []
for x in notImg :

    src = join(x[0], x[1], x[2])
    ext = os.path.splitext(x[2])[1]
    new_name = x[1] + add_file_name + ext

    sn = 0
    while os.path.exists(join(dst_dir, new_name)) :
        sn += 1
        new_name = x[1] + add_file_name +" " + str(sn) + ext
    try :
        os.rename(src, join(dst_dir, new_name))
        success.append([src, join(dst_dir, new_name)])
    except Exception as e:
        fail.append([src, e.__class__, e])

print(f"이동 {len(success)}개, 실패{len(fail)}개")

이동 242개, 실패1개


In [16]:
print(fail)

[['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류\\20498815 이갑선 원인서류\\0220141270521_신용대출신청서_0001_61923136.pdf', <class 'FileNotFoundError'>, FileNotFoundError(2, '지정된 파일을 찾을 수 없습니다')]]


In [30]:
f = "L41805056190001_0001.jpg"
m = re.search(r"(?<=_)\d{4,4}", f).group()
m

'0001'

### img2pdf 단일 폴더 내 파일 합치기

In [38]:
# # 폴더내 이미지 파일 하나로 합치기. path에 생성
# ############################################
# path = r"D:\구 스캔파일\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류\20428545"
# ############################################
# p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
# add_file_name = path.split(os.path.sep)[-1] + " 원인서류 여타매각사자료포함"

# img_list = [join(path, f) for f in os.listdir(path) if p_extension.search(f)]
# output = join(path, add_file_name + ".pdf")
# if os.path.exists(output) :
#     sn = 0
#     while os.path.exists(output) :
#         sn += 1
#         output = join(path, add_file_name + str(sn) + ".pdf")

# with open(output,"wb") as f:
#     f.write(img2pdf.convert(img_list))

KeyError: 262